介绍自动微分

In [2]:
import torch
x = torch.arange(4.0)
x


tensor([0., 1., 2., 3.])

一个标量函数关于向量的梯度是向量，并且与具有相同的形状

In [ ]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)启用梯度追踪
x.grad  # 默认值是None

In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

调用反向传播函数来自动计算y关于x每个分量的梯度

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

<pre>
当x改变时，再次得到梯度  
    当执行 y = 2 * torch.dot(x, x) 时，PyTorch 会记录x到y的计算路径。如果后续直接修改x的值（如 x[0] = 5），  
而没有重新构建计算图，backward()可能会基于旧的计算图（包含原始 x 的值）计算梯度，导致错误  
    安全做法：  
        重新创建 x（如 x = torch.tensor(...)），或  
        通过 x.data 修改值(谨慎使用)，或  
        在 torch.no_grad() 上下文中修改。  
    始终记得清空梯度（x.grad.zero_()），避免累加。  
注意：  
    每次都要重新定义y的表达值
<pre>


In [12]:
x = torch.tensor([5., 1., 2., 3.], requires_grad=True)  # 直接用新值重建 x
y = 2 * torch.dot(x, x)
y.backward()
x.grad

tensor([20.,  4.,  8., 12.])

In [13]:
# 通过 x.data 修改值（避免影响计算图）
x.grad.zero_()
x.data[0] = 5  # 修改底层数据，不破坏计算图
y = 2 * torch.dot(x, x)  # 重新计算 y
y.backward()
x.grad

tensor([20.,  4.,  8., 12.])

In [14]:
# 在无梯度追踪的上下文中修改 x
with torch.no_grad():
    x[0] = 10
x.grad.zero_()
y = 2 * torch.dot(x, x)
y.backward()
x.grad

tensor([40.,  4.,  8., 12.])

如果需要多次计算梯度，建议将逻辑封装为函数或循环：

In [10]:
def compute_gradient(x):
    if x.grad is not None:
        x.grad.zero_()
    y = 2 * torch.dot(x, x)
    y.backward()
    return x.grad

# 初始计算
x = torch.arange(4.0, requires_grad=True)
print("第一次梯度:", compute_gradient(x))

# 修改 x 并重新计算
x.data.add_(1)
print("修改后梯度:", compute_gradient(x))

第一次梯度: tensor([ 0.,  4.,  8., 12.])
修改后梯度: tensor([ 4.,  8., 12., 16.])


当y关于x的表达式发生变化时需要先梯度清零

In [15]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

 对于高阶和高维的y和x，求导的结果可以是一个高阶张量  
 backward()调用对象需要是标量  
 好在机器学习里只是计算样本批量的梯度之和，再作平均  
 因此只需对y进行求和变成标量即可  
 

In [16]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([20.,  2.,  4.,  6.])

y.backward()需要传入的的参数  
当y是张量（多维）时，必须提供一个与y形状相同的gradient参数，指定如何将 y 的每个分量加权求和得到标量，才能反向传播。  
y是标量（单个值）时，可以直接调用 y.backward()（等价于 y.backward(torch.tensor(1.0))  
$X=\left\{ x_{ij} \right\}_{3 \times 4}$  
$A=\left\{ a_{ij} \right\}_{3 \times 3}$  
$Y=\left\{ X^T\cdot A\cdot X \right\}_{4 \times 4}$  
$Y=sum(diag(Y))=tr(X^T\cdot A\cdot X)$  
$\left. \frac{dY}{dX} \right. =\left\{(A+A^T)X \right\}_{3 \times 4}$  
$\left. \frac{dY}{dA} \right. =(X\cdot X^T)_{3 \times 3}$  

In [37]:
X=torch.arange(12, dtype=torch.float64).reshape(3,4)
A=torch.randn(3,3, dtype=torch.float64,requires_grad=True)
X.requires_grad_(True)
Y=torch.mm(torch.mm(X.T,A),X)
gradient=torch.eye(4,4, dtype=torch.float64)
Y.backward(gradient=gradient)
DY_DX=torch.mm((A+A.T),X)
DY_DA=torch.mm(X,X.T)
X,A,Y,gradient
print(X.grad,'\n',DY_DX)
print(A.grad,'\n',DY_DA)

tensor([[10.4428, 13.0896, 15.7364, 18.3831],
        [-2.6402, -3.5598, -4.4794, -5.3989],
        [-8.8983, -8.5755, -8.2527, -7.9298]], dtype=torch.float64) 
 tensor([[10.4428, 13.0896, 15.7364, 18.3831],
        [-2.6402, -3.5598, -4.4794, -5.3989],
        [-8.8983, -8.5755, -8.2527, -7.9298]], dtype=torch.float64,
       grad_fn=<MmBackward0>)
tensor([[ 14.,  38.,  62.],
        [ 38., 126., 214.],
        [ 62., 214., 366.]], dtype=torch.float64) 
 tensor([[ 14.,  38.,  62.],
        [ 38., 126., 214.],
        [ 62., 214., 366.]], dtype=torch.float64, grad_fn=<MmBackward0>)


分离计算  
$y=f(x)$  
$z=h(x,y)=h(x,f(x))=g(x)$  
计算$\frac{dz}{dx}= \frac{\partial h}{\partial x}+ \frac{\partial h}{\partial f} \times \frac{\partial f}{\partial x}$  
$\left. \frac{dz}{dx} \right|_{y}=\left. \frac{\partial h}{\partial x}+
 \frac{\partial h}{\partial f} \right|_{y} \times \left. \frac{\partial f}{\partial x} \right|_{y}$  
希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用,即令  
$\left. \frac{\partial h}{\partial f} \right|_{y}=0$  从而计算$\left. \frac{dz}{dx} \right|_{y}=\left. \frac{\partial h}{\partial x} \right.$  

In [38]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u


tensor([True, True, True, True])

还可以单独计算$\frac{dy}{dx}$

In [39]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使是非常复杂的函数代码，也可以求梯度  
并且分段函数(一阶导数不连续)也可以求，因为梯度是针对某一个特定点求出来的

In [40]:
def f(a):
    """ f(a)=d*a,d取决于a """
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a


tensor(True)